In [1]:
import pandas as pd
import swifter
import numpy as np
import re

In [2]:
df = pd.read_parquet('USCS Codes Keywords 1000 topics 1gram.parquet')

In [43]:
term_value_cols = [x for x in df.columns if 'term_value' in x]
df[term_value_cols] = df[term_value_cols].astype(float)

In [4]:
df.head()

,chapter_id,chapter_number,chapter_header,section_id,section_number,section_header,section_text,term_0,term_value_0,term_1,...,term_25,term_value_25,term_26,term_value_26,term_27,term_value_27,term_28,term_value_28,term_29,term_value_29
0,/us/usc/t46/stI/ch1,1,DEFINITIONS,/us/usc/t46/s101,101,Agency,"In this title, the term “agency” means a depar...",united states government,0.2923447501607981,agency,...,None,None,None,None,None,None,None,None,None,None
1,/us/usc/t46/stI/ch1,1,DEFINITIONS,/us/usc/t46/s102,102,Barge,"In this title, the term “barge” means a non-se...",barge,0.3082300928838998,term,...,None,None,None,None,None,None,None,None,None,None
2,/us/usc/t46/stI/ch1,1,DEFINITIONS,/us/usc/t46/s103,103,Boundary Line,"In this title, the term “Boundary Line” means ...",boundary line,0.30017653592988647,line,...,None,None,None,None,None,None,None,None,None,None
3,/us/usc/t46/stI/ch1,1,DEFINITIONS,/us/usc/t46/s104,104,Citizen of the United States,"In this title, the term “citizen of the United...",united states,0.2514064404411455,nationality act,...,None,None,None,None,None,None,None,None,None,None
4,/us/usc/t46/stI/ch1,1,DEFINITIONS,/us/usc/t46/s105,105,Consular officer,"In this title, the term “consular officer” mea...",consular officer,0.2971596257078336,united states government,...,None,None,None,None,None,None,None,None,None,None


In [5]:
term_columns = [x for x in df.columns if ('term' in x) & ('value' not in x)]

In [6]:
term_columns

['term_0',
 'term_1',
 'term_2',
 'term_3',
 'term_4',
 'term_5',
 'term_6',
 'term_7',
 'term_8',
 'term_9',
 'term_10',
 'term_11',
 'term_12',
 'term_13',
 'term_14',
 'term_15',
 'term_16',
 'term_17',
 'term_18',
 'term_19',
 'term_20',
 'term_21',
 'term_22',
 'term_23',
 'term_24',
 'term_25',
 'term_26',
 'term_27',
 'term_28',
 'term_29']

In [13]:
keyword_set = df[term_columns].apply(lambda x: set(x.dropna().values), axis=1)

In [14]:
keyword_set.head()

0    {term, department, united states government, a...
1                         {vessel, term, barge, title}
2    {boundary line, u.s.c., act, b, term, line, fe...
3    {u.s.c., individual, nationality act, term, na...
4    {officer, employee, term, united states govern...
dtype: object

In [15]:
keyword_set = keyword_set.to_frame()
keyword_set.columns = ['keywords']

In [105]:
df['keywords'] = keyword_set['keywords']

In [20]:
keyword_set['age'] = keyword_set['keywords'].apply(lambda x: 'age' in x)
keyword_set['discrimination'] = keyword_set['keywords'].apply(lambda x: 'discrimination' in x)
keyword_set['age_discrimination'] = keyword_set['keywords'].apply(lambda x: 'age discrimination' in x)

In [23]:
keyword_set['age_discrimination'].value_counts()

False    40646
True         7
Name: age_discrimination, dtype: int64

In [36]:
age_and_discrimination = keyword_set.loc[((keyword_set['age'].values) & (keyword_set['discrimination'])) | (keyword_set['age_discrimination'].values)]

In [37]:
age_and_discrimination

,keywords,age,discrimination,age_discrimination
2221,"{title ix, activities, discrimination, attorne...",True,True,False
2725,"{title ix, activities, discrimination, attorne...",True,True,False
3881,"{section e–b(a, discrimination, religion, basi...",True,True,False
5454,"{activities, economic status, other organizati...",True,True,False
5789,"{basis, activities, chapter, federal financial...",True,True,False
5790,"{activity, program, person, basis, b, particip...",True,True,False
7774,"{hawaiian homes commission act, discrimination...",True,True,False
15607,"{code, equal employment opportunity commission...",True,True,False
16188,"{amendment, report, examination, congress, par...",True,False,True
16195,"{ancillary benefits, equal employment opportun...",False,False,True


In [38]:
age_discrimination = keyword_set[keyword_set['age_discrimination'].values]
age_discrimination

,keywords,age,discrimination,age_discrimination
16188,"{amendment, report, examination, congress, par...",True,False,True
16195,"{ancillary benefits, equal employment opportun...",False,False,True
16198,"{equal employment opportunity commission, semi...",True,True,True
16669,"{application, real wages, commercialization, u...",False,False,True
18604,"{employing office, covered employee, such act,...",True,False,True
44961,"{legislative branch, u.s.c. e et seq, disabili...",False,False,True
44976,"{such study, government printing office, relat...",False,False,True


In [39]:
for x in age_and_discrimination['keywords'].values:
    print(x)
    print()

{'title ix', 'activities', 'discrimination', 'attorney general', 'appropriate civil action', 'programs', 'religion', 'u.s.c.', 'basis', 'federal financial assistance', 'sex', 'age', 'civil rights act', 'title vi', 'section', 'et seq', 'functions', 'education amendments', 'age discrimination act', 'race', 'u.s.c. d et seq', 'handicap', 'such other action', 'national origin', 'ground', 'person', 'powers', 'part', 'rehabilitation act', 'color'}

{'title ix', 'activities', 'discrimination', 'attorney general', 'appropriate civil action', 'programs', 'religion', 'u.s.c.', 'basis', 'federal financial assistance', 'sex', 'age', 'civil rights act', 'title vi', 'section', 'et seq', 'funds', 'education amendments', 'age discrimination act', 'race', 'u.s.c. d et seq', 'subchapter', 'handicap', 'such other action', 'national origin', 'ground', 'person', 'part', 'rehabilitation act', 'color'}

{'section e–b(a', 'discrimination', 'religion', 'basis', 'purpose', 'practice', 'sections', 'sex', 'age', 

In [40]:
df.loc[age_and_discrimination.index]

,chapter_id,chapter_number,chapter_header,section_id,section_number,section_header,section_text,term_0,term_value_0,term_1,...,term_25,term_value_25,term_26,term_value_26,term_27,term_value_27,term_28,term_value_28,term_29,term_value_29
2221,/us/usc/t42/ch6A,6A,PUBLIC HEALTH SERVICE,/us/usc/t42/s300w–7,300w–7,Nondiscrimination provisions,For the purpose of applying the prohibitions ...,u.s.c. d et seq,0.2075581441477576,age discrimination act,...,religion,0.013284587282284406,ground,0.011688002211878054,person,0.010709729367871306,powers,0.010091371874625997,functions,0.009924972961344597
2725,/us/usc/t42/ch7,7,SOCIAL SECURITY,/us/usc/t42/s708,708,Nondiscrimination provisions,For the purpose of applying the prohibitions ...,u.s.c. d et seq,0.2058220715779273,age discrimination act,...,subchapter,0.012554952799613089,part,0.012021345023483452,ground,0.011576809520107789,person,0.010659199472915227,funds,0.010156261583376779
3881,/us/usc/t42/ch21,21,CIVIL RIGHTS,/us/usc/t42/s2000e–16a,2000e–16a,Short title; purpose; definition,Sections e–a to e–c of this title may be cited...,government employee rights act,0.09982574946816734,national origin,...,None,None,None,None,None,None,None,None,None,None
5454,/us/usc/t42/ch68,68,DISASTER RELIEF,/us/usc/t42/s5151,5151,Nondiscrimination in disaster assistance,"The President shall issue, and may alter and a...",such other regulations,0.10625410448902636,assistance activities,...,chapter,0.020432016343343306,grounds,0.020393182856313502,disability,0.018594764635383863,discrimination,0.018223943779516697,site,0.017031587466120268
5789,/us/usc/t42/ch76,76,AGE DISCRIMINATION IN FEDERALLY ASSISTED PROGRAMS,/us/usc/t42/s6101,6101,Statement of purpose,It is the purpose of this chapter to prohibit ...,federal financial assistance,0.2662115892672679,discrimination,...,None,None,None,None,None,None,None,None,None,None
5790,/us/usc/t42/ch76,76,AGE DISCRIMINATION IN FEDERALLY ASSISTED PROGRAMS,/us/usc/t42/s6102,6102,Prohibition of discrimination,Pursuant to regulations prescribed under secti...,federal financial assistance,0.26496445924219086,discrimination,...,None,None,None,None,None,None,None,None,None,None
7774,/us/usc/t42/ch130,130,NATIONAL AFFORDABLE HOUSING,/us/usc/t42/s12832,12832,Nondiscrimination,No person in the United States shall on the gr...,age discrimination act,0.15998673727181512,hawaiian homes commission act,...,lands,0.01657496666885904,use,0.01349691882900237,part,0.012542588928414219,connection,0.012090674953968381,secretary,0.011650461837960655
15607,/us/usc/t26/stF/ch76,76,JUDICIAL PROCEEDINGS,/us/usc/t26/s7471,7471,Employees,\n \n \n \n \n \n prohibit discrimination on ...,united states code,0.13316642186519811,merit systems protection board,...,individual,0.015252255615290661,action,0.015193083901668753,employment,0.015007056991882213,basis,0.01288492364014681,complaints,0.011232059230675655
16188,/us/usc/t29/ch14,14,AGE DISCRIMINATION IN EMPLOYMENT,/us/usc/t29/s624,624,Study by Secretary of Labor; reports to Presid...,an examination of the effect of the amendment...,upper age limitation,0.13240883148223198,age discrimination,...,amendment,0.015754595820015274,d,0.014949599410746072,secretary,0.014932747847408568,congress,0.014021607901465014,None,None
16195,/us/usc/t29/ch14,14,AGE DISCRIMINATION IN EMPLOYMENT,/us/usc/t29/s631,631,Age limits,\n \n Nothing in this chapter shall be constr...,such retirement benefit,0.17580075008438714,immediate nonforfeitable annual retirement ben...,...,savings,0.016746646881233312,pension,0.01653959996654679,-year period,0.01615123092486278,profit-sharing,0.015355132018670385,subsec,0.014168365319775923


In [27]:
for x in age_discrimination['keywords'].values:
    print(x)
    print()

{'amendment', 'report', 'examination', 'congress', 'paragraph', 'study', 'contract', 'such limitation', 'age', 'upper age limitation', 'interim report', 'section', 'employment act amendments', 'd', 'other arrangement', 'age discrimination', 'feasibility', 'title', 'effect', 'final form', 'exemption', 'determination', 'president', 'secretary', 'years', 'certain executive employees', 'c', 'teaching personnel', 'subsection'}

{'ancillary benefits', 'equal employment opportunity commission', 'such act', 'retirement', 'rollover contributions', 'subsec', 'compensation plan', '-year period', 'such plans', 'plan', 'employment act', 'such retirement benefit', 'compulsory retirement', 'high policymaking position', 'such benefit', 'savings', 'benefit', 'section', 'such employee', 'employee', 'pension', 'age discrimination', 'employees', 'such plan', 'immediate nonforfeitable annual retirement benefit', 'retirement benefit test', 'straight life annuity', 'profit-sharing', 'subsection', 'bona fide 

In [28]:
df.loc[age_discrimination.index]

,chapter_id,chapter_number,chapter_header,section_id,section_number,section_header,section_text,term_0,term_value_0,term_1,...,term_25,term_value_25,term_26,term_value_26,term_27,term_value_27,term_28,term_value_28,term_29,term_value_29
16188,/us/usc/t29/ch14,14,AGE DISCRIMINATION IN EMPLOYMENT,/us/usc/t29/s624,624,Study by Secretary of Labor; reports to Presid...,an examination of the effect of the amendment...,upper age limitation,0.13240883148223198,age discrimination,...,amendment,0.015754595820015274,d,0.014949599410746072,secretary,0.014932747847408568,congress,0.014021607901465014,None,None
16195,/us/usc/t29/ch14,14,AGE DISCRIMINATION IN EMPLOYMENT,/us/usc/t29/s631,631,Age limits,\n \n Nothing in this chapter shall be constr...,such retirement benefit,0.17580075008438714,immediate nonforfeitable annual retirement ben...,...,savings,0.016746646881233312,pension,0.01653959996654679,-year period,0.01615123092486278,profit-sharing,0.015355132018670385,subsec,0.014168365319775923
16198,/us/usc/t29/ch14,14,AGE DISCRIMINATION IN EMPLOYMENT,/us/usc/t29/s633a,633a,Nondiscrimination on account of age in Federal...,\n be responsible for the review and evaluati...,equal employment opportunity commission,0.18654557282059478,employment act amendments,...,nondiscrimination,0.01763025147828265,operation,0.01721491081813496,title,0.01702770792877741,complaints,0.01606394910171966,acceptance,0.015748160784235934
16669,/us/usc/t29/ch29,29,WORKERS TECHNOLOGY SKILL DEVELOPMENT,/us/usc/t29/s2701,2701,Findings,"In an increasingly competitive world economy,...",advanced workplace technologies,0.07963186656512355,united states house,...,real wages,0.016897510568476132,age discrimination,0.016853948763792273,higher education,0.01678815791055291,development,0.016140015277147295,commercialization,0.016124519864743548
18604,/us/usc/t3/ch5,5,EXTENSION OF CERTAIN RIGHTS AND PROTECTIONS TO...,/us/usc/t3/s411,411,Rights and protections under title VII of the ...,"race, color, religion, sex, or national origi...",federal advisory committee act,0.0862792020242818,such act,...,employment,0.01770704817967722,executive branch,0.0163805992522736,a(b)()(d,0.01629958421798795,age,0.015501312564424708,president,0.014513918345566326
44961,/us/usc/t2/ch24,24,CONGRESSIONAL ACCOUNTABILITY,/us/usc/t2/s1302,1302,Application of laws,The Fair Labor Standards Act of U.S.C. et ...,u.s.c. e et seq,0.18502586504746169,et seq,...,effective date,0.016541024427302643,labor-management relations,0.015812592666518143,employment,0.01566613602735979,provision,0.015068086748062086,worker adjustment,0.014887514022596336
44976,/us/usc/t2/ch24,24,CONGRESSIONAL ACCOUNTABILITY,/us/usc/t2/s1371,1371,Study and recommendations regarding General Ac...,the General Accounting Office; the Governmen...,u.s.c. e et seq,0.17479575863690108,et seq,...,section,0.024686633932032923,board,0.020086093959091132,speaker,0.018761667251785404,study,0.018432316677268253,congress,0.01770137814690137


In [29]:
for x in df.loc[age_discrimination.index,'section_text'].values:
    print(x)
    print('------------------------------------')
    print()

 an examination of the effect of the amendment made by section (a) of the Age Discrimination in Employment Act Amendments of  in raising the upper age limitation established by section (a) of this title to  years of age;  a determination of the feasibility of eliminating such limitation;  a determination of the feasibility of raising such limitation above  years of age; and  an examination of the effect of the exemption contained in section (c) of this title, relating to certain executive employees, and the exemption contained in section (d) of this title, relating to tenured teaching personnel.  The Secretary may undertake the study required by paragraph () of this subsection directly or by contract or other arrangement.  The report required by subsection (a) of this section shall be transmitted to the President and to the Congress as an interim report not later than , and in final form not later than .
------------------------------------


 
  Nothing in this chapter shall be constr

In [45]:
eeo_section = df[df['section_number'].str.contains('2000e')]

In [46]:
eeo_section.describe()

,term_value_0,term_value_1,term_value_2,term_value_3,term_value_4,term_value_5,term_value_6,term_value_7,term_value_8,term_value_9,...,term_value_20,term_value_21,term_value_22,term_value_23,term_value_24,term_value_25,term_value_26,term_value_27,term_value_28,term_value_29
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,21.000000,21.000000,20.000000,20.000000,...,18.000000,17.000000,17.000000,17.000000,17.000000,15.000000,12.000000,12.000000,12.000000,12.000000
mean,0.152803,0.125760,0.101322,0.090983,0.079027,0.065301,0.062698,0.056954,0.051589,0.048464,...,0.023004,0.022344,0.021440,0.020121,0.019021,0.018230,0.018474,0.017697,0.016566,0.015757
std,0.058027,0.054897,0.046171,0.041677,0.031584,0.021545,0.020068,0.015074,0.014411,0.014373,...,0.006847,0.005741,0.005418,0.005639,0.006013,0.006531,0.006432,0.006642,0.006349,0.006207
min,0.075534,0.055638,0.055318,0.055005,0.039945,0.033625,0.033034,0.031110,0.027573,0.026621,...,0.006394,0.014856,0.014188,0.013602,0.012364,0.009249,0.010568,0.007813,0.007264,0.007176
25%,0.102390,0.087655,0.075326,0.063797,0.059044,0.050238,0.046328,0.044774,0.041879,0.038404,...,0.020226,0.017695,0.016809,0.015742,0.013863,0.013565,0.014068,0.013856,0.013035,0.012343
50%,0.133672,0.109754,0.090716,0.080617,0.077866,0.063600,0.065992,0.061611,0.050034,0.043961,...,0.021147,0.020132,0.019780,0.017782,0.017283,0.016184,0.016644,0.016049,0.015371,0.014545
75%,0.186703,0.152530,0.120809,0.100902,0.086451,0.078869,0.071601,0.067155,0.062814,0.060547,...,0.029978,0.028032,0.025782,0.023760,0.021432,0.022934,0.022332,0.021828,0.019262,0.019069
max,0.278830,0.275318,0.267176,0.245509,0.177895,0.122529,0.113879,0.079807,0.076912,0.076430,...,0.032699,0.032390,0.031649,0.030835,0.030293,0.030012,0.029450,0.029241,0.029154,0.028173


In [47]:
eeo_section[eeo_section['term_value_0'] > 0.15]

,chapter_id,chapter_number,chapter_header,section_id,section_number,section_header,section_text,term_0,term_value_0,term_1,...,term_25,term_value_25,term_26,term_value_26,term_27,term_value_27,term_28,term_value_28,term_29,term_value_29
3868,/us/usc/t42/ch21,21,CIVIL RIGHTS,/us/usc/t42/s2000e–4,2000e–4,Equal Employment Opportunity Commission,", That assignment, removal, and compensation o...",technical assistance training institute,0.199721,training revolving fund,...,other entity,0.017404,entity,0.014307,subsection,0.014233,monies,0.013286,current expenditures,0.012365
3871,/us/usc/t42/ch21,21,CIVIL RIGHTS,/us/usc/t42/s2000e–7,2000e–7,Effect on State laws,Nothing in this subchapter shall be deemed to ...,political subdivision,0.178023,unlawful employment practice,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
3873,/us/usc/t42/ch21,21,CIVIL RIGHTS,/us/usc/t42/s2000e–9,2000e–9,Conduct of hearings and investigations pursuan...,For the purpose of all hearings and investigat...,commission,0.201882,investigations,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
3875,/us/usc/t42/ch21,21,CIVIL RIGHTS,/us/usc/t42/s2000e–11,2000e–11,Veterans’ special rights or preference,Nothing contained in this subchapter shall be ...,local law,0.250952,special rights,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
3878,/us/usc/t42/ch21,21,CIVIL RIGHTS,/us/usc/t42/s2000e–14,2000e–14,Equal Employment Opportunity Coordinating Coun...,The Equal Employment Opportunity Commission sh...,equal employment opportunity commission,0.181427,equal employment opportunity legislation,...,october,0.016184,agreements,0.015584,year,0.015228,report,0.015133,congress,0.012669
3879,/us/usc/t42/ch21,21,CIVIL RIGHTS,/us/usc/t42/s2000e–15,2000e–15,Presidential conferences; acquaintance of lead...,"The President shall, as soon as feasible after...",equal employment opportunity,0.188461,employment agencies,...,obligations,0.015122,None,NaN,None,NaN,None,NaN,None,NaN
3880,/us/usc/t42/ch21,21,CIVIL RIGHTS,/us/usc/t42/s2000e–16,2000e–16,Employment by Federal Government,\n be responsible for the annual review and a...,equal employment opportunity program,0.278830,regional equal employment opportunity plan,...,approval,0.013439,experience,0.013352,recommendations,0.012724,organizations,0.012283,allocation,0.012280
3884,/us/usc/t42/ch21,21,CIVIL RIGHTS,/us/usc/t42/s2000e–17,2000e–17,"Procedure for denial, withholding, termination...","No Government contract, or portion thereof, wi...",affirmative action plan,0.172677,such plan,...,portion,0.009249,None,NaN,None,NaN,None,NaN,None,NaN
3921,/us/usc/t42/ch21E,21E,PRIVACY AND CIVIL LIBERTIES PROTECTION AND OVE...,/us/usc/t42/s2000ee–3,2000ee–3,Federal agency data mining reporting,This section may be cited as the “Federal Agen...,federal agency data mining reporting act,0.252630,agency,...,payment,0.013691,annex,0.010568,subparagraph,0.010408,section,0.008826,c,0.007733


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [85]:
tfidf = TfidfVectorizer(ngram_range=(1,1), max_df=0.6, min_df=2, max_features=100000, stop_words=["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"])

In [86]:
tfidf.fit(df['section_text'])

/home/jameslittiebrant/anaconda3/envs/keyword/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:394: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['articl', 'mon'] not in stop_words.
  warnings.warn(


TfidfVectorizer(max_df=0.6, max_features=100000, min_df=2,
                stop_words=['0o', '0s', '3a', '3b', '3d', '6b', '6o', 'a', 'a1',
                            'a2', 'a3', 'a4', 'ab', 'able', 'about', 'above',
                            'abst', 'ac', 'accordance', 'according',
                            'accordingly', 'across', 'act', 'actually', 'ad',
                            'added', 'adj', 'ae', 'af', 'affected', ...])

In [87]:
section_vecs = tfidf.transform(df['section_text'])

In [88]:
age_discrimination_vec = tfidf.transform(['age discrimination'])

In [89]:
from sklearn.metrics.pairwise import cosine_similarity

In [90]:
query_results = cosine_similarity(age_discrimination_vec, section_vecs)

In [91]:
query_results.shape

(1, 40653)

In [92]:
# query_results = query_results.todense()
df['query_values'] = query_results.T

In [93]:
df['query_values'].describe()

count    40653.000000
mean         0.001799
std          0.015453
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.569059
Name: query_values, dtype: float64

In [108]:
tfidf_similarity = df[df['query_values']>0].sort_values(by=['query_values'])

In [109]:
tfidf_similarity.loc[age_discrimination.index]

,chapter_id,chapter_number,chapter_header,section_id,section_number,section_header,section_text,term_0,term_value_0,term_1,...,term_26,term_value_26,term_27,term_value_27,term_28,term_value_28,term_29,term_value_29,query_values,keywords
16188,/us/usc/t29/ch14,14,AGE DISCRIMINATION IN EMPLOYMENT,/us/usc/t29/s624,624,Study by Secretary of Labor; reports to Presid...,an examination of the effect of the amendment...,upper age limitation,0.132409,age discrimination,...,d,0.014950,secretary,0.014933,congress,0.014022,None,NaN,0.356436,"{amendment, report, examination, congress, par..."
16195,/us/usc/t29/ch14,14,AGE DISCRIMINATION IN EMPLOYMENT,/us/usc/t29/s631,631,Age limits,\n \n Nothing in this chapter shall be constr...,such retirement benefit,0.175801,immediate nonforfeitable annual retirement ben...,...,pension,0.016540,-year period,0.016151,profit-sharing,0.015355,subsec,0.014168,0.157130,"{ancillary benefits, equal employment opportun..."
16198,/us/usc/t29/ch14,14,AGE DISCRIMINATION IN EMPLOYMENT,/us/usc/t29/s633a,633a,Nondiscrimination on account of age in Federal...,\n be responsible for the review and evaluati...,equal employment opportunity commission,0.186546,employment act amendments,...,operation,0.017215,title,0.017028,complaints,0.016064,acceptance,0.015748,0.369610,"{equal employment opportunity commission, semi..."
16669,/us/usc/t29/ch29,29,WORKERS TECHNOLOGY SKILL DEVELOPMENT,/us/usc/t29/s2701,2701,Findings,"In an increasingly competitive world economy,...",advanced workplace technologies,0.079632,united states house,...,age discrimination,0.016854,higher education,0.016788,development,0.016140,commercialization,0.016125,0.108849,"{application, real wages, commercialization, u..."
18604,/us/usc/t3/ch5,5,EXTENSION OF CERTAIN RIGHTS AND PROTECTIONS TO...,/us/usc/t3/s411,411,Rights and protections under title VII of the ...,"race, color, religion, sex, or national origi...",federal advisory committee act,0.086279,such act,...,executive branch,0.016381,a(b)()(d,0.016300,age,0.015501,president,0.014514,0.208338,"{employing office, covered employee, such act,..."
44961,/us/usc/t2/ch24,24,CONGRESSIONAL ACCOUNTABILITY,/us/usc/t2/s1302,1302,Application of laws,The Fair Labor Standards Act of U.S.C. et ...,u.s.c. e et seq,0.185026,et seq,...,labor-management relations,0.015813,employment,0.015666,provision,0.015068,worker adjustment,0.014888,0.111889,"{legislative branch, u.s.c. e et seq, disabili..."
44976,/us/usc/t2/ch24,24,CONGRESSIONAL ACCOUNTABILITY,/us/usc/t2/s1371,1371,Study and recommendations regarding General Ac...,the General Accounting Office; the Governmen...,u.s.c. e et seq,0.174796,et seq,...,board,0.020086,speaker,0.018762,study,0.018432,congress,0.017701,0.116786,"{such study, government printing office, relat..."


In [110]:
tfidf_similarity.loc[age_and_discrimination.index].sort_values(by=['query_values'])

,chapter_id,chapter_number,chapter_header,section_id,section_number,section_header,section_text,term_0,term_value_0,term_1,...,term_26,term_value_26,term_27,term_value_27,term_28,term_value_28,term_29,term_value_29,query_values,keywords
16669,/us/usc/t29/ch29,29,WORKERS TECHNOLOGY SKILL DEVELOPMENT,/us/usc/t29/s2701,2701,Findings,"In an increasingly competitive world economy,...",advanced workplace technologies,0.079632,united states house,...,age discrimination,0.016854,higher education,0.016788,development,0.016140,commercialization,0.016125,0.108849,"{application, real wages, commercialization, u..."
44961,/us/usc/t2/ch24,24,CONGRESSIONAL ACCOUNTABILITY,/us/usc/t2/s1302,1302,Application of laws,The Fair Labor Standards Act of U.S.C. et ...,u.s.c. e et seq,0.185026,et seq,...,labor-management relations,0.015813,employment,0.015666,provision,0.015068,worker adjustment,0.014888,0.111889,"{legislative branch, u.s.c. e et seq, disabili..."
44976,/us/usc/t2/ch24,24,CONGRESSIONAL ACCOUNTABILITY,/us/usc/t2/s1371,1371,Study and recommendations regarding General Ac...,the General Accounting Office; the Governmen...,u.s.c. e et seq,0.174796,et seq,...,board,0.020086,speaker,0.018762,study,0.018432,congress,0.017701,0.116786,"{such study, government printing office, relat..."
16195,/us/usc/t29/ch14,14,AGE DISCRIMINATION IN EMPLOYMENT,/us/usc/t29/s631,631,Age limits,\n \n Nothing in this chapter shall be constr...,such retirement benefit,0.175801,immediate nonforfeitable annual retirement ben...,...,pension,0.016540,-year period,0.016151,profit-sharing,0.015355,subsec,0.014168,0.157130,"{ancillary benefits, equal employment opportun..."
18553,/us/usc/t47/ch16,16,BROADBAND ACCESS,/us/usc/t47/s1726,1726,General provisions,"No individual in the United States may, on the...",assistant secretary,0.061343,first fiscal year,...,amounts,0.016133,age,0.015315,oversight,0.013982,disability,0.013593,0.159810,"{commerce, department, discrimination, religio..."
5454,/us/usc/t42/ch68,68,DISASTER RELIEF,/us/usc/t42/s5151,5151,Nondiscrimination in disaster assistance,"The President shall issue, and may alter and a...",such other regulations,0.106254,assistance activities,...,grounds,0.020393,disability,0.018595,discrimination,0.018224,site,0.017032,0.161734,"{activities, economic status, other organizati..."
15607,/us/usc/t26/stF/ch76,76,JUDICIAL PROCEEDINGS,/us/usc/t26/s7471,7471,Employees,\n \n \n \n \n \n prohibit discrimination on ...,united states code,0.133166,merit systems protection board,...,action,0.015193,employment,0.015007,basis,0.012885,complaints,0.011232,0.169013,"{code, equal employment opportunity commission..."
18604,/us/usc/t3/ch5,5,EXTENSION OF CERTAIN RIGHTS AND PROTECTIONS TO...,/us/usc/t3/s411,411,Rights and protections under title VII of the ...,"race, color, religion, sex, or national origi...",federal advisory committee act,0.086279,such act,...,executive branch,0.016381,a(b)()(d,0.016300,age,0.015501,president,0.014514,0.208338,"{employing office, covered employee, such act,..."
16665,/us/usc/t29/ch28,28,FAMILY AND MEDICAL LEAVE,/us/usc/t29/s2651,2651,Effect on other laws,Nothing in this Act or any amendment made by t...,national origin,0.118261,state law,...,None,NaN,None,NaN,None,NaN,None,NaN,0.251840,"{act, basis, race, discrimination, state, sex,..."
3881,/us/usc/t42/ch21,21,CIVIL RIGHTS,/us/usc/t42/s2000e–16a,2000e–16a,Short title; purpose; definition,Sections e–a to e–c of this title may be cited...,government employee rights act,0.099826,national origin,...,None,NaN,None,NaN,None,NaN,None,NaN,0.267261,"{section e–b(a, discrimination, religion, basi..."


In [116]:
tfidf_sorted_query_w_keywords = tfidf_similarity.loc[age_and_discrimination.index].sort_values(by=['query_values'])

In [117]:
for i, x in enumerate(tfidf_sorted_query_w_keywords['keywords'].values):
    print(i)
    print(x)
    print()

0
{'application', 'real wages', 'commercialization', 'united states house', 'federal technology programs', 'worker participation', 'worker', 'worker organizations', 'rapid development', 'unprecedented competitive challenge', 'competitive world economy', 'information technology workplace', 'states', 'private sector', 'advanced technologies', 'advanced workplace technologies', 'united states', 'development', 'private high-tech workforce training programs', 'implementation', 'age discrimination', 'workplace', 'worker organization participation', 'living standards', 'high living standards', 'higher education', 'skill level', 'nonprofit organizations', 'private companies', 'economic growth'}

1
{'legislative branch', 'u.s.c. e et seq', 'disabilities act', 'representatives', 'provisions', 'employment act', 'u.s.c.', 'congressional record', 'amendment note', 'employee polygraph protection act', 'civil rights act', 'such provisions', 'title vii', 'et seq', 'provision', 'house', 'labor-manageme

In [124]:
tfidf_sorted_query_w_keywords.iloc[1]

chapter_id                                          /us/usc/t2/ch24
chapter_number                                                   24
chapter_header                         CONGRESSIONAL ACCOUNTABILITY
section_id                                         /us/usc/t2/s1302
section_number                                                 1302
                                        ...                        
term_value_28                                              0.015068
term_29                                           worker adjustment
term_value_29                                              0.014888
query_values                                               0.111889
keywords          {legislative branch, u.s.c. e et seq, disabili...
Name: 44961, Length: 69, dtype: object

In [123]:
tfidf_sorted_query_w_keywords.iloc[0]['section_text']

' In an increasingly competitive world economy, the companies and nations that lead in the rapid development, commercialization, and application of new and advanced technologies, and in the high-quality, competitively priced production of goods and services, will lead in economic growth, employment, and high living standards.  While the United States remains the world leader in science and invention, it has not done well in rapidly making the transition from achievement in its research laboratories to high-quality, competitively priced production of goods and services. This lag and the unprecedented competitive challenge that the United States has faced from abroad have contributed to a drop in real wages and living standards.  Companies that are successfully competitive in the rapid development, commercialization, application, and implementation of advanced technologies, and in the successful delivery of goods and services, recognize that worker participation and labor-management coop